# Cluster Evaluation

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
DATA_PATH = Path(os.getenv("DATA_PATH"))


# only for .ipynb because relative imports don't work
root_path = Path(DATA_PATH).parent
os.chdir(str(root_path))

import src.database.db_connector as db

In [ ]:
# database name for results
db_name = "clustering_db"
cnx = db.connect_to_database(db_name)
# cursor = db.get_connection_cursor(cnx)

## Get list of top n labels for each cluster

In [ ]:
# top n labels for each cluster withj 
top_n_query = """
    SELECT rj.ranked_k_value, rj.cluster_id, rj.label_name, rj.factor_tf_idf
    FROM ( 
        SELECT * FROM (
            SELECT 
            j.ranked_k_value,
            j.cluster_id, 
            @cluster_rank := if(@current_cluster = j.cluster_id, @cluster_rank  + 1, 1) AS cluster_rank ,
            j.label_name,
            j.factor_tf_idf,
            @current_cluster := j.cluster_id
            FROM ( 
                SELECT  *
                FROM matchings AS m
                INNER JOIN ( 
                    SELECT ma.k_value as ranked_k_value
                    FROM matchings AS ma
                    WHERE (ma.run_name = "v01_all_run2_pca90" OR ma.run_name = "v01_all_run3_pca90") 
                    group by ma.k_value
                ) as ranked_k on ranked_k.ranked_k_value = m.k_value
                WHERE (m.run_name = "v01_all_run2_pca90" OR m.run_name = "v01_all_run3_pca90")  AND m.k_value = ranked_k.ranked_k_value
                ORDER BY ranked_k.ranked_k_value, m.cluster_id, m.factor_tf_idf desc
            ) as j
        ) as ranked_clusters
        where ranked_clusters.cluster_rank <= 5
    ) as rj;
"""

df_topn = pd.read_sql(top_n_query, cnx)
print(df_topn)



## Scores over top *n* values of each cluster

In [ ]:
scores_query = """
    SELECT rj.ranked_k_value, 
    AVG(rj.cluster_label_count),
    AVG(rj.cluster_size),
    AVG(rj.factor_tf_idf),
    STD(rj.factor_tf_idf)
    FROM ( 
        SELECT 
        ranked_clusters.cluster_id, 
        ranked_clusters.ranked_k_value, 
        c.n_screenshots as cluster_size,
        ranked_clusters.cluster_label_count,
        ranked_clusters.percentage,
        ranked_clusters.factor,
        ranked_clusters.tf,
        ranked_clusters.idf,
        ranked_clusters.tf_idf,
        ranked_clusters.factor_tf_idf
        FROM (
            SELECT 
            j.ranked_run,
            j.cluster_id,
            j.ranked_k_value, 
            @cluster_rank := if(@current_cluster = j.cluster_id, @cluster_rank  + 1, 1) AS cluster_rank ,
            j.label_name,
            j.cluster_label_count,
            j.percentage,
            j.factor,
            j.tf,
            j.idf,
            j.tf_idf,
            j.factor_tf_idf,
            @current_cluster := j.cluster_id
            FROM ( 
                SELECT  *
                FROM matchings AS m
                INNER JOIN ( 
                    SELECT ma.k_value as ranked_k_value, ma.run_name as ranked_run
                    FROM matchings AS ma
                    WHERE (ma.run_name = "v01_all_run2_pca90" OR ma.run_name = "v01_all_run3_pca90") 
                    group by ma.k_value
                ) as ranked_k on ranked_k.ranked_k_value = m.k_value
                INNER JOIN labels as l on m.label_name = l.name
                WHERE (m.run_name = "v01_all_run2_pca90" OR m.run_name = "v01_all_run3_pca90")  AND m.k_value = ranked_k.ranked_k_value AND l.type !="technology"
                ORDER BY ranked_k.ranked_k_value, m.cluster_id, m.factor_tf_idf desc
            ) as j
        ) as ranked_clusters
        inner join clusters as c on c.k_value = ranked_clusters.ranked_k_value and c.cluster_id = ranked_clusters.cluster_id and ranked_clusters.ranked_run = c.run_name
        where ranked_clusters.cluster_rank <= 5
    ) as rj
    GROUP BY rj.ranked_k_value;
"""

df_scores = pd.read_sql(scores_query, cnx)
print(df_scores)

## Plot `factor_tf_idf` and cluster size regarding *k*

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(18,6))

x = df_scores["ranked_k_value"]
y_fac = df_scores["AVG(rj.factor_tf_idf)"]
error = np.array(df_scores["STD(rj.factor_tf_idf)"])

df_scores.plot(x="ranked_k_value", y="AVG(rj.factor_tf_idf)", label="Factor-LF-ILF", legend=True, xlim=(10,200), ylim=(0.0, 1.0), ax=ax, xlabel="k")
ax.set_xticks(df_scores["ranked_k_value"])

# avg factor_tf_id with std as error zone
ax.fill_between(x, y_fac - (error/2), y_fac + (error/2), alpha=0.1)

# cluster size
df_scores.plot(x="ranked_k_value", y="AVG(rj.cluster_size)", secondary_y=True, label="Cluster size", legend=True, ax=ax, xlabel="k")

# add grid to x-axis
ax.xaxis.grid(True)

## IDF scores for each k

In [ ]:
op_query = """
    select b.k, avg(b.idf_s) FROM
(
	Select *,  idf(a.uniq_cnt, a.k) as idf_s 
	FROM (
		SELECT rj.ranked_k_value as k, rj.cluster_id, rj.label_name, count(rj.label_name) as uniq_cnt
		FROM ( 
			SELECT * FROM (
				SELECT 
				j.ranked_k_value,
				j.cluster_id, 
				@cluster_rank := if(@current_cluster = j.cluster_id, @cluster_rank  + 1, 1) AS cluster_rank ,
				j.label_name,
				j.factor_tf_idf,
				@current_cluster := j.cluster_id
				FROM ( 
					SELECT  *
					FROM matchings AS m
					INNER JOIN ( 
						SELECT ma.k_value as ranked_k_value
						FROM matchings AS ma
						WHERE (ma.run_name = "v01_all_run2_pca90" OR ma.run_name = "v01_all_run3_pca90") 
						group by ma.k_value
					) as ranked_k on ranked_k.ranked_k_value = m.k_value
					WHERE (m.run_name = "v01_all_run2_pca90" OR m.run_name = "v01_all_run3_pca90")  AND m.k_value = ranked_k.ranked_k_value
					ORDER BY ranked_k.ranked_k_value, m.cluster_id, m.factor_tf_idf desc
				) as j
			) as ranked_clusters
			where ranked_clusters.cluster_rank > 0
		) as rj
		GROUP BY rj.ranked_k_value, rj.label_name
	) as a
	ORDER BY a.k
) as b
Group by b.k;
"""

df_op = pd.read_sql(op_query, cnx)
print(df_op)

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(18,6))

x = df_op["k"]
y_fac = df_op["avg(b.idf_s)"]
# error = np.array(df_scores["STD(rj.factor_tf_idf)"])

df_op.plot(x="k", y="avg(b.idf_s)", label="avg(b.idf_s)", legend=True, xlim=(10,200), ylim=(0.0, 2.0), ax=ax)
ax.set_xticks(df_op["k"])

# avg factor_tf_id with std as error zone
# ax.fill_between(x, y_fac - (error/2), y_fac + (error/2), alpha=0.1)

# cluster size
df_scores.plot(x="ranked_k_value", y="AVG(rj.cluster_size)", secondary_y=True, label="cluster size", legend=True, ax=ax)

# add grid to x-axis
ax.xaxis.grid(True)

## Unique Labels Ratio for each k in top 2

In [ ]:
unique_label_ratios = []

for k_value in range(10, 201, 5):

    k_query = f"""
        SELECT rj.label_name
        FROM ( 
            SELECT * FROM (
                SELECT 
                j.ranked_k_value,
                j.cluster_id, 
                @cluster_rank := if(@current_cluster = j.cluster_id, @cluster_rank  + 1, 1) AS cluster_rank ,
                j.label_name,
                j.factor_tf_idf,
                @current_cluster := j.cluster_id
                FROM ( 
                    SELECT  *
                    FROM matchings AS m
                    INNER JOIN ( 
                        SELECT ma.k_value as ranked_k_value
                        FROM matchings AS ma
                        WHERE (ma.run_name = "v01_all_run2_pca90" OR ma.run_name = "v01_all_run3_pca90") AND ma.k_value={str(k_value)} 
                        group by ma.k_value
                    ) as ranked_k on ranked_k.ranked_k_value = m.k_value
                    WHERE (m.run_name = "v01_all_run2_pca90" OR m.run_name = "v01_all_run3_pca90")  AND m.k_value = ranked_k.ranked_k_value
                    ORDER BY ranked_k.ranked_k_value, m.cluster_id, m.factor_tf_idf desc
                ) as j
            ) as ranked_clusters
            where ranked_clusters.cluster_rank <= 5
        ) as rj;
    """

    k_df = pd.read_sql(k_query, cnx)
    unique_count = len(np.unique(k_df.values))
    total_count = len(k_df.values)
    unique_label_ratios.append((k_value,  (unique_count / total_count)))


fig, ax = plt.subplots(figsize=(18,6))

x = [tpl[0] for tpl in unique_label_ratios]
y = [tpl[1] for tpl in unique_label_ratios]

plt.xticks(x)
plt.ylim(0,1)
plt.plot(x,y)


## WSS + Sil Score

In [ ]:
import src.clustering.kmeans as km

In [ ]:


args = [
    "v01_all",
    "wss_sil_eval_1",
    "10",
    "200",
    "5",
    "0.9"
]

list_k, sse, sil = km.run_training(args)
print(list_k)

In [ ]:
# store sse + sil in .csv file
df_kmeans = pd.DataFrame({"k": list_k, "sse": sse, "sil": sil})
print(df_kmeans)

df_kmeans.to_csv(DATA_PATH / "extracted_features" / "clustering_sse_wss.csv")

In [ ]:
print(sse)
fig, ax = plt.subplots(figsize=(18,6))
plt.xticks(list_k)
plt.xlabel("k")
plt.ylabel("Sum of Squared Errors")
plt.gca().xaxis.grid(True)
plt.plot(list_k,sse)

In [ ]:
fig, ax = plt.subplots(figsize=(18,6))
plt.xticks(list_k)
plt.xlabel("k")
plt.ylabel("Silhoette Score")
plt.gca().xaxis.grid(True)
plt.plot(list_k,sil)

## Cluster Calculate Real IDF

In [ ]:
cluster_count_query = """
    SELECT m.label_name, Count(m.cluster_id) as cluster_count FROM clustering_db.matchings m
    inner join labels l on m.label_name=l.name
    where m.k_value=%s and l.type != "technology"
    group by m.label_name;
"""

In [ ]:
avgs = []
stds = []

for k in range(10, 201, 5):
    # count cluster occurrences for each label (term)
    df_cc_k = pd.read_sql(cluster_count_query, cnx, params=[k])

    # calculate average occurrences + std error
    avg_cc = df_cc_k["cluster_count"].mean()
    std_cc = df_cc_k["cluster_count"].std()
    
    avgs.append(avg_cc)
    stds.append(std_cc)

print(avgs)
print(stds)

In [ ]:
import math

fig, ax = plt.subplots(figsize=(12,6))

# k's
x = list(range(10,201,5))

# 1 + idf score based on average count
y = [ math.log10(k / elem) for elem, k in zip(avgs, x)]

# upper bound error: 1 + idf(avg + std)
error_upper = np.array([math.log10(k / (a + s)) for s, a, k in zip(stds, avgs, x)])

# lower bound error: 1 + idf(avg - std)
error_lower = np.array([math.log10(k/ (a - s)) for s, a, k in zip(stds, avgs, x)])


# plot error zone
plt.fill_between(x, error_upper, error_lower, alpha=0.1)

plt.xlabel("k")
plt.ylabel("ICF")

plt.xticks([10,25,50,75,100,125,150,175,200])
plt.ylim(-0.2,0.2)
plt.gca().xaxis.grid(True)
plt.plot(x,y)